In [306]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [307]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
# proxy = {'http':'thomasttam%40gmail.com:phaw2colt_ec.LUK@au621.nordvpn.com',
#         'https':'thomasttam%40gmail.com:phaw2colt_ec.LUK@au621.nordvpn.com'}

# .get response code
r = requests.get('https://www.amazon.com/s?k=board+games&page=1', headers=headers)
content = r.content
soup = BeautifulSoup(content)

In [308]:
soup

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://completion.amazon.com" rel="dns-prefetch"/>
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a

# Get Item Information from Search Results

For each search result page, scrape the general information such as board board name, the number of reviews, the average rating, and the price.

In [309]:
def get_search_results_items(no_pages):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    alls = []
    # scrape i number of pages
    for i in range(1, no_pages+1):
        r = requests.get('https://www.amazon.com/s?k=board+games&page='+str(i), headers=headers)
        content = r.content
        soup = BeautifulSoup(content)

        # for each board game...scrape the asin(amazon product #), name, price, rating, # of ratings
        for d in soup.findAll('div', attrs={'class':'sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item s-asin sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32'}):
            name = d.find('span', attrs={'class':'a-size-base-plus a-color-base a-text-normal'})
            price = d.find('span', attrs={'class':'a-offscreen'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
            no_of_ratings = d.find('span', attrs={'class':'a-size-base'})
            asin = d['data-asin']

            # append information into a list
            all1=[]
            
            if asin is not None:
                all1.append(asin)
            else:
                all1.append('-')
            
            if name is not None:
                all1.append(name.text)
            else:
                all1.append("unknown-product")

            if price is not None:
                all1.append(price.text)
            else:
                all1.append('$0')

            if rating is not None:
                all1.append(rating.text)
            else:
                all1.append('-')
                
            if no_of_ratings is not None:
                all1.append(no_of_ratings.text)
            else:
                all1.append('-') 
                

            # for each item, append all1 list into alls list
            alls.append(all1)  

    return alls

In [310]:
# # convert list of lists into dataframe
# items_df = pd.DataFrame(get_search_results_items(10),columns=['asin_id',
#                                        'name',
#                                        'price',
#                                        'avg_rating',
#                                        'no_of_ratings'])

In [311]:
# items_df.to_csv('csv_files_test/items_df.csv')

## Items - Preprocessing

In [312]:
items_df = pd.read_csv('csv_files_test/items_df.csv',index_col=0).rename({'rating':'avg_rating'},axis=1)

In [313]:
items_df

,asin_id,name,price,avg_rating,no_of_ratings
0,B076HK9H7Z,Sorry! Game,$0,4.7 out of 5 stars,"7,555"
1,B08GD242TJ,Hasbro Gaming Guess Who? Game Original Guessin...,$9.97,4.9 out of 5 stars,30
2,B00C0ULS3G,Battleship Classic Board Game Strategy Game Ag...,$0,4.7 out of 5 stars,"4,151"
3,B08636S37Z,Funko Disney The Haunted Mansion – Call of The...,$24.97,4.7 out of 5 stars,84
4,B07B7KS87Q,Game Of Life,$19.99,4.7 out of 5 stars,"4,284"
...,...,...,...,...,...
301,B08GG9J8HR,Coatl - Strategy Board Game for 1-4 Players,$39.99,4.8 out of 5 stars,7
302,B07M5BMVS3,CLUE: Scooby Doo! Board Game | Official Scooby...,$39.95,4.7 out of 5 stars,341
303,B07HM7WZT9,Rio Grande Games Power Grid Recharged,$35.70,4.8 out of 5 stars,132
304,B083Q2FKTN,Escape Room The Game Expansion Pack – The Brea...,$11.99,5.0 out of 5 stars,1


### Remove duplicates

In [314]:
items_df = items_df.drop_duplicates(subset=['asin_id'])

### Convert price to float values

In [315]:
# convert price column as float values

# strip '?' symbol from price
items_df['price'] = items_df['price'].apply(lambda x:x.strip('$'))

# convert price column to float
items_df['price'] = items_df['price'].astype('float')

# check if column was converted to float values
items_df['price']

/Users/thomasttam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/thomasttam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0       0.00
1       9.97
2       0.00
3      24.97
4      19.99
       ...  
300    14.95
301    39.99
302    39.95
303    35.70
304    11.99
Name: price, Length: 252, dtype: float64

### Clean rating column by removing unnecessary characters

In [316]:
items_df['avg_rating'].value_counts()

4.8 out of 5 stars    68
4.7 out of 5 stars    51
4.6 out of 5 stars    32
-                     21
4.5 out of 5 stars    19
4.4 out of 5 stars    18
5.0 out of 5 stars    14
4.9 out of 5 stars    12
4.3 out of 5 stars     8
4.2 out of 5 stars     3
4.1 out of 5 stars     3
4.0 out of 5 stars     1
3.8 out of 5 stars     1
3.9 out of 5 stars     1
Name: avg_rating, dtype: int64

In [317]:
# remove items with '-' rating
items_df = items_df[items_df['avg_rating'] != '-'].reset_index(drop=True)

In [318]:
# get the number rating from the rating column by taking the left characters of string
items_df['avg_rating'] = items_df['avg_rating'].apply(lambda x: x[0:3])
# convert column to float vlaues
items_df['avg_rating'] = items_df['avg_rating'].astype('float')

### Convert no_of_rating into int values

In [319]:
# replace commas with empty space
items_df['no_of_ratings'] = items_df['no_of_ratings'].apply(lambda x: x.replace(',',''))

# # strip characters with empty spaces such as ' by '
items_df['no_of_ratings'] = items_df['no_of_ratings'].apply(lambda x: x.strip())

# # once stripped, remove items with '-' rating and 'by' rating
items_df = items_df[(items_df['no_of_ratings'] != '-') & (items_df['no_of_ratings'] != 'by')].reset_index(drop=True)

# # cast column as float values
items_df['no_of_ratings'] = items_df['no_of_ratings'].astype('int')

In [320]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   asin_id        228 non-null    object 
 1   name           228 non-null    object 
 2   price          228 non-null    float64
 3   avg_rating     228 non-null    float64
 4   no_of_ratings  228 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 9.0+ KB


In [321]:
items_df

,asin_id,name,price,avg_rating,no_of_ratings
0,B076HK9H7Z,Sorry! Game,0.00,4.7,7555
1,B08GD242TJ,Hasbro Gaming Guess Who? Game Original Guessin...,9.97,4.9,30
2,B00C0ULS3G,Battleship Classic Board Game Strategy Game Ag...,0.00,4.7,4151
3,B08636S37Z,Funko Disney The Haunted Mansion – Call of The...,24.97,4.7,84
4,B07B7KS87Q,Game Of Life,19.99,4.7,4284
...,...,...,...,...,...
223,B01C5U4PTY,"Rubik's Race Game, Head To Head Fast Paced Squ...",14.95,4.5,3981
224,B08GG9J8HR,Coatl - Strategy Board Game for 1-4 Players,39.99,4.8,7
225,B07M5BMVS3,CLUE: Scooby Doo! Board Game | Official Scooby...,39.95,4.7,341
226,B07HM7WZT9,Rio Grande Games Power Grid Recharged,35.70,4.8,132


In [322]:
# save as csv file
items_df.to_csv('csv_files_test/cleaned_items_df.csv')

# Get Reviews for each Item

In [323]:
def get_reviews(asin_no, no_review_pages):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    alls = []
    # for each asin, scrape i number of pages
    for i in range(1, no_review_pages+1):
        r = requests.get('https://www.amazon.com/product-reviews/'+asin_no+'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber='+str(i), headers=headers)
        content = r.content
        soup = BeautifulSoup(content)

        # scrape the following for each page
        for d in soup.findAll('div', attrs={'class':'a-section review aok-relative'}):
            product_id = d['id']
            review_title = d.find('a', attrs={'class':'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold'})
            review_description = d.find('span', attrs={'class':'a-size-base review-text review-text-content'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
#             user_name = d.find('span', attrs={'class': 'a-profile-name'}).text

            all1=[]
            
            all1.append(asin_no)
            
            if product_id is not None:
                all1.append(product_id)
            else:
                all1.append("unknown-product")

            if review_title is not None:
                all1.append(review_title.text)
            else:
                all1.append('$0')

            if review_description is not None:
                #print(rating.text)
                all1.append(review_description.text)
            else:
                all1.append('-1')

            if rating is not None:
                #print(price.text)
                all1.append(rating.text)
            else:
                all1.append('0')


            alls.append(all1)  

    return alls

In [324]:
# get_reviews('B08636S37Z',2)

## Scrape on several occasions using different VPN to prevent slamming the server (October 13, 2020)

In [325]:
# # scraper 1
# # append reviews from all items to a list

# reviews = []

# for id_no in items_df['asin_id'][0:100]:
#     for row in get_reviews(id_no,3):
#         reviews.append(row)

In [326]:
# len(reviews)

In [327]:
# # scraper 2
# # append reviews from all items to a list

# reviews1 = []

# for id_no in items_df['asin_id'][101:150]:
#     for row in get_reviews(id_no,3):
#         reviews1.append(row)

In [328]:
# len(reviews1)

In [329]:
# # scraper 3
# # append reviews from all items to a list

# reviews2 = []

# for id_no in items_df['asin_id'][151:200]:
#     for row in get_reviews(id_no,3):
#         reviews2.append(row)

In [330]:
# len(reviews2)

In [331]:
# # scraper 4
# # append reviews from all items to a list

# reviews3 = []

# for id_no in items_df['asin_id'][201:250]:
#     for row in get_reviews(id_no,3):
#         reviews3.append(row)

In [332]:
# len(reviews3)

In [333]:
# # scraper 5
# # append reviews from all items to a list

# reviews4 = []

# for id_no in items_df['asin_id'][251:304]:
#     for row in get_reviews(id_no,3):
#         reviews4.append(row)

In [334]:
# len(reviews4)

### Combine all reviews into Dataframe

In [335]:
# reviews_df = pd.DataFrame(reviews + reviews1 + reviews2 + reviews3 + reviews4,columns=['asin_id',
#                                                                                        'review_id',
#                                                                                        'review_title',
#                                                                                        'review_description',
#                                                                                        'user_rating'])

In [336]:
# reviews_df.to_csv('csv_files_test/reviews_df.csv')

## Preprocessing - Reviews

In [337]:
reviews_df = pd.read_csv('csv_files_test/reviews_df.csv',index_col=0)

In [338]:
reviews_df.head()

,asin_id,review_id,review_title,review_description,user_rating
0,B076HK9H7Z,R1OSPWS88F2CUZ,DO NOT BUY!!!,I would give this zero stars if I could! If ...,1.0
1,B076HK9H7Z,R1DCFJ8VYSN17B,Is this the millennial version?,This is not the original sorry game. It only...,1.0
2,B076HK9H7Z,R1V07N4GXA7RSL,Wimp and Crybaby Edition,We bought this to replace our old Sorry game...,1.0
3,B076HK9H7Z,R2Z262NZDEU2EY,NOT the original/regular Sorry!,Be warned that this is not the sorry you gre...,2.0
4,B076HK9H7Z,RG3XIFV1PUX9Y,"Not the classic by a long shot, but okay.","Definitely not the classic game, with only 3...",4.0


In [339]:
reviews_df.shape

(6693, 5)

### scraping didn't lead to great results - price was in review_title so will remove them (186 rows)

In [340]:
reviews_df = reviews_df[reviews_df['review_title'] != '$0']
reviews_df.head()

,asin_id,review_id,review_title,review_description,user_rating
0,B076HK9H7Z,R1OSPWS88F2CUZ,DO NOT BUY!!!,I would give this zero stars if I could! If ...,1.0
1,B076HK9H7Z,R1DCFJ8VYSN17B,Is this the millennial version?,This is not the original sorry game. It only...,1.0
2,B076HK9H7Z,R1V07N4GXA7RSL,Wimp and Crybaby Edition,We bought this to replace our old Sorry game...,1.0
3,B076HK9H7Z,R2Z262NZDEU2EY,NOT the original/regular Sorry!,Be warned that this is not the sorry you gre...,2.0
4,B076HK9H7Z,RG3XIFV1PUX9Y,"Not the classic by a long shot, but okay.","Definitely not the classic game, with only 3...",4.0


### Parse user rating and convert to float values

In [341]:
reviews_df['user_rating'].value_counts()

5.0    4301
4.0     887
1.0     657
3.0     517
2.0     331
Name: user_rating, dtype: int64

In [342]:
# # get the number rating from the review_rating column by taking the left characters of string
# reviews_df['user_rating'] = reviews_df['user_rating'].apply(lambda x: x[0:3])
# # convert column to float vlaues
# reviews_df['user_rating'] = reviews_df['user_rating'].astype('float')
# reviews_df

### Clean the '/n' from review_title and review_description

In [343]:
# strip '/n' characters from title and description
reviews_df['review_title'] = reviews_df['review_title'].apply(lambda x:x.strip('\n'))

# was getting an error when stripping review_description due to N/A (row 56)
reviews_df['review_description'][reviews_df['review_description'].isna()]
reviews_df['review_description'] = reviews_df['review_description'].fillna('-1')
reviews_df['review_description'] = reviews_df['review_description'].apply(lambda x:x.strip('\n'))

In [344]:
reviews_df.head()

,asin_id,review_id,review_title,review_description,user_rating
0,B076HK9H7Z,R1OSPWS88F2CUZ,DO NOT BUY!!!,I would give this zero stars if I could! If ...,1.0
1,B076HK9H7Z,R1DCFJ8VYSN17B,Is this the millennial version?,This is not the original sorry game. It only...,1.0
2,B076HK9H7Z,R1V07N4GXA7RSL,Wimp and Crybaby Edition,We bought this to replace our old Sorry game...,1.0
3,B076HK9H7Z,R2Z262NZDEU2EY,NOT the original/regular Sorry!,Be warned that this is not the sorry you gre...,2.0
4,B076HK9H7Z,RG3XIFV1PUX9Y,"Not the classic by a long shot, but okay.","Definitely not the classic game, with only 3...",4.0


In [345]:
reviews_df.shape

(6693, 5)

### Remove duplicate reviews

In [346]:
reviews_df = reviews_df.drop_duplicates(subset=['review_id'])

In [347]:
reviews_df.shape

(5539, 5)

### save processed reviews_df as csv

In [348]:
reviews_df.to_csv('csv_files_test/cleaned_reviews_df.csv')

# Get Additional Information from each Item Page

In [351]:
def get_prod_details(asin_no):
    
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    # get response code
    r = requests.get('https://www.amazon.com/dp/'+asin_no, headers=headers)
    content = r.content
    soup = BeautifulSoup(content)

    prod_details = []

    prod_details.append(asin_no)
    
    product_description = soup.select('#productDescription')[0].p.get_text().strip()
#     product_description = soup.find_all('div',attrs={'id':'productDescription'})[0].text.strip()
    prod_details.append(product_description)

    table = soup.find('table', attrs={'class':'a-keyvalue prodDetTable'})

    for line in table.findAll('tr'):

        if line.find('th').text.strip() == 'Product Dimensions' or line.find('th').text.strip() == 'Package Dimensions':
            prod_details.append(line.find('td').text.strip())

        if line.find('th').text.strip() == 'Item Weight':
            prod_details.append(line.find('td').text.strip())

        if line.find('th').text.strip() == 'Manufacturer recommended age':
            prod_details.append(line.find('td').text.strip())

        if line.find('th').text.strip() == 'Manufacturer':
            prod_details.append(line.find('td').text.strip())
            
    return prod_details


In [357]:
item_descriptions = []

for asin_id in items_df['asin_id']:
    try:
        item_descriptions.append(get_prod_details(asin_id))
    except:
        pass

In [358]:
item_descriptions

[['B076HK9H7Z',
  "Slide, collide and score to win the Sorry! Game. Kids draw cards to see how far they get to move one of the pawns on the board. If they land on a Slide they can zip to the end and bump their opponents' pawns – or their own! Jump over pawns and hide in the Safety zone where opponents can't go. Kids keep on moving and bumping until they get all three of their pawns from Start to Home. But watch out, because if the pawn gets bumped, Sorry! It's all the way back to Start! Sorry! and all related characters are trademarks of Hasbro.",
  '1.61 x 10.51 x 10.51 inches',
  '1 pounds',
  '6 years and up',
  'Hasbro'],
 ['B08GD242TJ',
  "Bring back childhood memories and introduce “the original guessing game” to your kids and grandkids. With the Guess Who? game, each player chooses a mystery character card and then using yes or no questions, they try to figure out the other player's mystery person. This edition features character art and most character names not seen in other ed

In [359]:
pd.DataFrame(item_descriptions)

,0,1,2,3,4,5
0,B076HK9H7Z,"Slide, collide and score to win the Sorry! Gam...",1.61 x 10.51 x 10.51 inches,1 pounds,6 years and up,Hasbro
1,B08GD242TJ,Bring back childhood memories and introduce “t...,1.61 x 12.01 x 10.51 inches,1.35 pounds,6 years and up,Hasbro
2,B00C0ULS3G,This Battleship board game is the classic game...,2.87 x 10.51 x 10.51 inches,1.6 pounds,7 years and up,Hasbro
3,B08636S37Z,Disney’s beloved The Haunted Mansion comes to ...,2.5 x 10.5 x 10.5 inches,2.58 pounds,6 years and up,Funko
4,B07B7KS87Q,In The Game of Life game players choose a path...,1.63 x 15.75 x 10.5 inches,2 pounds,8 years and up,Hasbro
5,B00000DMF5,If you loved playing the Candy Land board game...,1.18 x 15.75 x 10.63 inches,1.12 pounds,3 months - 6 years,Hasbro
6,B01JYVHMVA,One murder… 6 suspects. In this suspenseful Cl...,1.61 x 10.51 x 10.51 inches,1 pounds,8 years and up,Hasbro
7,B00000IVAK,"It's fun, it's challenging, it's exciting, it'...",8.08 x 2.18 x 10.38 inches,7 ounces,7 years and up,Jax Ltd Inc
8,B00D7OAOYI,"Pop into the racing, chasing game where everyo...",1.61 x 10.51 x 10.51 inches,11.2 ounces,5 - 15 years,Hasbro
9,B013TO5PDU,Clue and all related characters are trademarks...,2.01 x 15.75 x 10.51 inches,2 pounds,8 years and up,Hasbro


## Preprocessing - Item Details

In [138]:
item_details_df

,asin_id,product_description,product_dimensions,item_weight,recommender_age,manufacturer
0,B076HK9H7Z,"Style:classic\n\n\nSlide, collide and score to...",1.61 x 10.51 x 10.51 inches,1 pounds,6 years and up,Hasbro
1,B00D7OAOYI,"Product Description\nPop into the racing, chas...",1.61 x 10.51 x 10.51 inches,11.2 ounces,5 - 15 years,Hasbro
2,B00C0ULS3G,Product Description\nThis Battleship board gam...,2.87 x 10.51 x 10.51 inches,1.6 pounds,7 years and up,Hasbro
3,B08636S37Z,Disney’s beloved The Haunted Mansion comes to ...,2.5 x 10.5 x 10.5 inches,2.58 pounds,6 years and up,Funko
4,B00000IVAK,"Product Description\nIt's fun, it's challengin...",8.08 x 2.18 x 10.38 inches,7 ounces,7 years and up,Jax Ltd Inc
5,B07B7KS87Q,In The Game of Life game players choose a path...,1.63 x 15.75 x 10.5 inches,2 pounds,8 years and up,Hasbro
6,B08GD242TJ,Bring back childhood memories and introduce “t...,1.61 x 12.01 x 10.51 inches,1.35 pounds,6 years and up,Hasbro
7,B01JYVHMVA,Style:Classic Clue\n\n\nOne murder… 6 suspects...,1.61 x 10.51 x 10.51 inches,1 pounds,8 years and up,Hasbro
8,B00A2HD40E,Product Description\nFour diseases have broken...,12 x 8.6 x 1.7 inches,2.3 pounds,13 - 15 years,Publisher Services Inc (PSI)
9,B088QSDB7S,Gloomhaven Jaws of the Lion is a fully stand a...,11.5 x 9.1 x 4.5 inches,6.34 pounds,14 years and up,Cepholafair Games


### Clean the '/n' from Product Description

In [139]:
item_details_df['product_description'] = item_details_df['product_description'].apply(lambda x: x.replace('\n',''))

In [206]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows', None)

In [141]:
item_details_df

,asin_id,product_description,product_dimensions,item_weight,recommender_age,manufacturer
0,B076HK9H7Z,"Style:classicSlide, collide and score to win t...",1.61 x 10.51 x 10.51 inches,1 pounds,6 years and up,Hasbro
1,B00D7OAOYI,"Product DescriptionPop into the racing, chasin...",1.61 x 10.51 x 10.51 inches,11.2 ounces,5 - 15 years,Hasbro
2,B00C0ULS3G,Product DescriptionThis Battleship board game ...,2.87 x 10.51 x 10.51 inches,1.6 pounds,7 years and up,Hasbro
3,B08636S37Z,Disney’s beloved The Haunted Mansion comes to ...,2.5 x 10.5 x 10.5 inches,2.58 pounds,6 years and up,Funko
4,B00000IVAK,"Product DescriptionIt's fun, it's challenging,...",8.08 x 2.18 x 10.38 inches,7 ounces,7 years and up,Jax Ltd Inc
5,B07B7KS87Q,In The Game of Life game players choose a path...,1.63 x 15.75 x 10.5 inches,2 pounds,8 years and up,Hasbro
6,B08GD242TJ,Bring back childhood memories and introduce “t...,1.61 x 12.01 x 10.51 inches,1.35 pounds,6 years and up,Hasbro
7,B01JYVHMVA,Style:Classic ClueOne murder… 6 suspects. In t...,1.61 x 10.51 x 10.51 inches,1 pounds,8 years and up,Hasbro
8,B00A2HD40E,Product DescriptionFour diseases have broken o...,12 x 8.6 x 1.7 inches,2.3 pounds,13 - 15 years,Publisher Services Inc (PSI)
9,B088QSDB7S,Gloomhaven Jaws of the Lion is a fully stand a...,11.5 x 9.1 x 4.5 inches,6.34 pounds,14 years and up,Cepholafair Games


In [142]:
# item_details = item_details.drop([54,59,65,70,91]).reset_index(drop=True)
# item_details.head()

### Parse item_weight and convert all values to ounces to keep consistency

In [143]:
item_details_df['item_weight'] = item_details_df['item_weight'].apply(lambda x: float(x[:-7]) * 16 
                                                                      if 'pounds' in x else float(x[:-7]))

In [145]:
item_details_df.to_csv('csv_files1/cleaned_item_details_df.csv')